In [2]:
#load data 

import pandas as pd

trip_data_file = "data/trip_data_4.csv"
trip_fare_file = "data/trip_fare_4.csv"

trips = pd.read_csv(trip_data_file, nrows=100000)
fares = pd.read_csv(trip_fare_file, nrows=100000)

full_tripdata = pd.merge(left=trips,right=fares, how='left', left_on=['medallion',' hack_license',' vendor_id',' pickup_datetime'], right_on=['medallion',' hack_license',' vendor_id',' pickup_datetime'])
full_tripdata = full_tripdata.drop_duplicates()

In [3]:
full_tripdata = full_tripdata.drop(columns=['medallion',' hack_license',' vendor_id',' rate_code',' store_and_fwd_flag'])

In [4]:
full_tripdata[" pickup_datetime"] = pd.to_datetime(full_tripdata[" pickup_datetime"])
full_tripdata[" dropoff_datetime"] = pd.to_datetime(full_tripdata[" dropoff_datetime"])

full_tripdata["weekday"] = full_tripdata[" pickup_datetime"].dt.dayofweek
full_tripdata["hourofday"] = full_tripdata[" pickup_datetime"].dt.hour

In [5]:
full_tripdata = full_tripdata.loc[(full_tripdata[' pickup_longitude'] != 0) & (full_tripdata[' pickup_latitude'] != 0) & (full_tripdata[' dropoff_latitude'] != 0) & (full_tripdata[' dropoff_latitude'] != 0)]

full_tripdata = full_tripdata.loc[(full_tripdata[' trip_distance'] != 0) & (full_tripdata[' trip_time_in_secs'] != 0)]

full_tripdata = full_tripdata.loc[(full_tripdata['weekday'] != 0) & (full_tripdata['hourofday'] != 0)]

In [6]:
import pickle
from sklearn.cluster import KMeans

model = pickle.load(open('location_cluster_kmean', 'rb'))

In [7]:
pickupLocations = full_tripdata.iloc[:,5:7]
full_tripdata["pickupLoc"] = model.predict(pickupLocations.as_matrix())

dropoffLocations = full_tripdata.iloc[:,7:9]
full_tripdata["dropoffLoc"] = model.predict(dropoffLocations.as_matrix())

In [8]:
training = full_tripdata.drop(columns=[' pickup_datetime',
                                       ' dropoff_datetime',
                                       ' trip_time_in_secs',
                                       ' trip_distance',
                                       ' dropoff_longitude',
                                       ' dropoff_latitude',
                                       ' surcharge',
                                       ' mta_tax',
                                       ' tolls_amount',
                                       ' total_amount',
                                       ' passenger_count',
                                       ' payment_type',
                                       ' pickup_longitude',
                                       ' pickup_latitude'])

In [9]:
training['weekday'] = training['weekday'].astype('category')
training['hourofday'] = training['hourofday'].astype('category')
training['pickupLoc'] = training['pickupLoc'].astype('category')
training['dropoffLoc'] = training['dropoffLoc'].astype('category')

In [10]:
X = training.iloc[:,2:]
Y_fare = training.iloc[:,0]
Y_tip = training.iloc[:,1]

X = pd.get_dummies(X)


In [11]:
recordNumber = 60000
X_cv = X.as_matrix()[0:recordNumber,:]
Y_fare_cv = Y_fare.tolist()[0:recordNumber]
Y_tip_cv = Y_tip.tolist()[0:recordNumber]

In [1]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score

model = MLPRegressor()
#scores4 = cross_val_score(model, X2, y, cv=4,scoring='neg_mean_squared_error',n_jobs=-1)

param_grid = [
 # {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
 # {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
  {'hidden_layer_sizes': [(20,20),(10,10),(20,30)]},
 ]

clf = GridSearchCV(model, param_grid, cv=2, scoring = 'r2'
                       n_jobs=-1)

clf.fit(X_cv,Y_fare_cv)

clf.cv_results_

SyntaxError: invalid syntax (<ipython-input-1-0f322e4206b1>, line 16)

In [13]:
clf.best_score_

0.7783934211604822

In [14]:
clf.best_params_


{'hidden_layer_sizes': (10, 10)}

In [15]:
#further choosing hyperparameters after decide on nn layer
model = MLPRegressor(hidden_layer_sizes=(10,10))

param_grid = [
  {'activation': [ 'logistic', 'tanh', 'relu'],'solver': [ 'lbfgs','sgd', 'adam'],'alpha': [ 0.0001, 0.001, 0.01]}
 ]

clf = GridSearchCV(model, param_grid, cv=2,
                       n_jobs=-1)

clf.fit(X_cv,Y_fare_cv)

print clf.cv_results_
#print clf.best_score_
#print clf.best_params_

{'std_train_score': array([0.02004691, 0.00251646, 0.01033075, 0.0140637 , 0.00633304,
       0.00716063, 0.02428988, 0.00717665, 0.01830945, 0.02306614,
       0.00394223, 0.00296378, 0.0120112 , 0.00257769, 0.00722974,
       0.03629763, 0.01123809, 0.00336738, 0.03506524, 0.00476358,
       0.00435693, 0.01227738, 0.00471875, 0.00615899, 0.00733193,
       0.01146674, 0.00935586]), 'rank_test_score': array([20, 18,  6, 24, 14, 11, 21,  5,  9, 22,  7,  3, 27, 13,  2, 26, 10,
       16, 25, 19,  1, 23, 12,  4, 17, 15,  8]), 'split1_train_score': array([0.761364  , 0.78851196, 0.83046579, 0.78518241, 0.7907525 ,
       0.82058044, 0.80862586, 0.80247439, 0.83879362, 0.70501607,
       0.87907781, 0.87925709, 0.68570992, 0.89098057, 0.8782573 ,
       0.67407026, 0.88833072, 0.88122797, 0.80032324, 0.90254965,
       0.85647425, 0.74212461, 0.89290637, 0.86553122, 0.82193522,
       0.89640433, 0.83628235]), 'std_score_time': array([0.02099991, 0.00699997, 0.00549996, 0.0250001 , 0.0029

In [16]:
print clf.best_params_
print clf.best_score_

{'alpha': 0.0001, 'activation': 'relu', 'solver': 'adam'}
0.7834990148618959


In [17]:
model

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10, 10), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)